In [ ]:
import os
import re
import json
import jsonlines
from tqdm import tqdm
import copy
#from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from datetime import datetime

from langchain.prompts import PromptTemplate 
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-02-01"

api_key_num = 2
if api_key_num == 1:
    os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://iegg-gpt-10.openai.azure.com/'
    os.environ["AZURE_OPENAI_API_KEY"] = "X"   
elif api_key_num == 2:
    os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://iegg-gpt-300.openai.azure.com'
    os.environ["AZURE_OPENAI_API_KEY"] = "X" 

llm_agent = AzureChatOpenAI(
    #deployment_name="iegg-gpt-4-turbo",
    model_name="gpt-4o",
    temperature= 0.7,
)

In [2]:
data_plot_folder = "../data/"
data_plot_file = data_plot_folder + "game_gen_12_14.jsonl"
story_dict_list = []
with open(data_plot_file, 'r', encoding='utf-8') as file:
    for line in file:
        # 将每行的 JSON 字符串解析为字典并添加到列表中
        story_dict = json.loads(line)
        story_dict_list.append(story_dict)
# print(json.dumps(story_dict_list[0],indent=2,ensure_ascii=False))
story_dict_list_new = copy.deepcopy(story_dict_list)

In [3]:
# 
# 这里应该是一个大循环
# 
# 
# 读取信息
# for story_dict_i in story_dict_list:
story_dict_i = story_dict_list[0]
Settings = story_dict_i["Settings"]
story_name = Settings["story_name"]
story_style = Settings["story_style"]
leading_role = Settings["leading_role"]
story_desc = Settings["story_desc"]
story_goal = Settings["story_goal"]
story_chars = Settings["story_chars"]
states = Settings["states"]
story_language = Settings["story_language"]
Episodes = story_dict_i["Episodes"]

print(json.dumps(Settings,indent=2,ensure_ascii=False))
print(json.dumps(Episodes[0],indent=2,ensure_ascii=False))

{
  "story_id": "",
  "story_name": "生化危机7(Resident Evil 7)",
  "story_style": [
    "恐怖生存",
    "心理惊悚",
    "黑暗氛围",
    "紧张刺激",
    "血腥暴力",
    "神秘悬疑",
    "压抑感",
    "家庭悲剧"
  ],
  "leading_role": "伊森·温特斯",
  "story_desc": "故事发生在美国路易斯安那州的一个偏远农场，这里是贝克家族的居所。贝克家族被一种神秘的力量所控制，变得极其暴力和疯狂。主角伊森·温特斯收到了一封来自失踪三年的妻子米娅的神秘信件，信中指引他前往这个农场寻找她。",
  "story_goal": "伊森·温特斯的主要目标是找到并救出他的妻子米娅，同时揭开贝克家族背后的恐怖秘密。在这个过程中，他需要在充满危险和恐怖的环境中生存下来，面对各种变异生物和疯狂的贝克家族成员，最终逃离这个恐怖的地方。",
  "story_chars": {
    "伊森·温特斯": "主角，普通的程序员，性格坚韧，勇敢且执着。在收到失踪三年的妻子米娅的求救信息后，前往路易斯安那州的贝克家族农场寻找她。在农场中，他经历了种种恐怖事件，与贝克家族成员展开了生死搏斗，最终成功救出妻子并逃离了农场。",
    "米娅·温特斯": "伊森的妻子，曾为一家神秘公司工作，性格坚强但内心深处有着不为人知的秘密。她在执行任务时被艾芙琳感染，导致失踪三年。被伊森找到后，她在贝克家族农场中经历了极大的痛苦和折磨，最终在伊森的帮助下恢复了理智并逃离了农场。",
    "杰克·贝克": "贝克家族的父亲，原本是一个普通的农场主，性格暴躁且残忍。在被艾芙琳感染后，他变得极其暴力和疯狂，频繁折磨和杀害入侵者。最终在与伊森的对抗中被击败。",
    "玛格丽特·贝克": "贝克家族的母亲，性格阴险且疯狂。在被艾芙琳感染后，她变得极其恐怖，拥有控制昆虫的能力。她对入侵者充满敌意，最终被伊森击败。",
    "卢卡斯·贝克": "贝克家族的儿子，性格狡猾且残忍，喜欢设计各种陷阱和游戏折磨他人。在被艾芙琳感染后，他的精神状态变得更加不稳定，最终被伊森击败。",
    "佐伊·贝克": "贝克家族的女儿，性格坚强且善良。在家人被

In [ ]:
prompt_evaluate_EpiSeq=PromptTemplate(
    template = """请根据总体章节的信息严格评价摘要剧情树是否合理，如不合理，请指出不合理的章节序号（只需指出不合理的章节），并说明不合理的原因，改进建议以及改进后的章节内容
        剧情是否连贯，完整，是否无缺失，是否有节奏问题和突兀之处，是否有不自然的中断，跳跃，拖沓; 
        不同章节之间不能出现重复内容；相邻的章节剧情应该连续但不相交，不要和相邻的章节叙述重复
        每个章节是否以主角{leading_role}来叙述

        剧情树的序号可能不连贯，仅是作为剧情的唯一标识
        你评估剧情时只需关注剧情树序号后的实际剧情
        如果改写过结局，那么改写的结局应该与之前出现过的结局有较大区别
        不能删除某一章节！如果某一章节和前面章节有冲突优先修改后续章节内容
        同时，你改进后的章节需要注意是否与其他章节或者新改写的章节冲突或者重复

        请严格寻找问题与不足，展示思考过程，用{language}回答
        输出使用如下的json格式:
        {{"(章节序号)":{{
            "问题以及": "(你的思考过程，不合理的原因)",
            "改进建议": "(如何修改这部分章节)",
            "改进后章节": "(改进后的章节内容)"
            }},
            ...(多个有问题的章节)
        }} 
        摘要剧情树如下: {episode_seq}; 之前出现过的结局如下: {other_ans}
        故事的总体风格如下: {story_style}; 故事的总体背景如下: {story_desc}; 故事的主要目标如下: {story_goal}; 故事的人物介绍如下: {story_chars}
    """,
    input_variables=["leading_role","language","episode_seq","story_style","story_desc","story_goal","story_chars","other_ans"],
)

In [ ]:
story_dict_new = copy.deepcopy(story_dict_i)
other_ans = ""
other_ans_num = []

# 评估连贯性
for j in tqdm(range(len(states))):
    state = {}
    state["seq"] = states[j]

    episode_seq = ""
    # 剧情序列
    # print(state)
    episode_seq += "\n".join(f"(episode {k}) {Episodes[k-1]['episode_desc']}\n" for k in state["seq"])
    other_ans_num.append(state["seq"][-1]) if not state["seq"][-1] in other_ans_num else None

    # 判断总体章节信息
    chain_evaluate_EpiSeq = prompt_evaluate_EpiSeq | llm_agent
    res_evaluate_EpiSeq = chain_evaluate_EpiSeq.invoke({"leading_role":leading_role,"language":story_language,"episode_seq":episode_seq,"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars,"other_ans":other_ans}).content
    if not state["seq"][-1] in other_ans_num:
        other_ans_num.append(state["seq"][-1])
        other_ans += "Other end: " + Episodes[state["seq"][-1]-1]["episode_desc"]
    try:
        cleaned_json_str1 = res_evaluate_EpiSeq.split('```json')[1].split('```')[0].strip()
    except:
        cleaned_json_str1 = res_evaluate_EpiSeq
    
    res1_dict = {}
    try:
        res1_dict = json.loads(cleaned_json_str1)
    except:
        print(cleaned_json_str1)
        res1_dict = json.loads(cleaned_json_str1)

    print(episode_seq)
    for key in res1_dict:
        pass
        key_num = key
        
        if type(key) == str:
            key_num = numbers = re.findall(r'\d+', key_num)
            key_num = [int(num) for num in key_num][0]

        print(key_num,Episodes[int(key_num)-1]["episode_desc"])
        print(json.dumps(res1_dict[key],indent=2,ensure_ascii=False))

        Episodes[int(key_num)-1]["episode_desc_old"] = Episodes[int(key_num)-1]["episode_desc"]
        Episodes[int(key_num)-1]["episode_desc"] = res1_dict[key]


    story_dict_new["Settings"]["states"][j] = state
    break

story_dict_list_new[0] = story_dict_new
#story_dict_list_new[i] = story_dict_new


  0%|          | 0/5 [01:04<?, ?it/s]

(episode 1) 伊森·温特斯的妻子米娅失踪已经三年了，突然有一天，他收到了一封来自她的求救信。信中，米娅告诉他，她被困在路易斯安那州的贝克家族农场。尽管心中充满疑虑和不安，伊森还是决定前往那个神秘而危险的地方，寻找他心爱的妻子。带着希望和恐惧，他踏上了这段未知的旅程。

(episode 2) 伊森踏入贝克家族的废弃农场，四周弥漫着腐朽的气息。他在一间阴暗的地下室里找到了失踪多年的妻子米娅，但她的行为异常，仿佛被某种力量控制。正当伊森试图带她逃离时，米娅突然发狂，攻击了他。伊森奋力抵抗，最终将她制服。然而，贝克家族的成员突然出现，将伊森打昏并拖走，黑暗再次笼罩了他的视线。

(episode 3) 伊森在地下室与米娅激烈搏斗后，筋疲力尽地瘫倒在地。突然，杰克·贝克出现，将他拖到主屋。伊森醒来时，发现自己被绑在椅子上，周围是贝克家族的成员，他们正进行一场诡异的晚餐。杰克粗暴地对待伊森，试图强迫他吃下不明食物。伊森设法挣脱束缚，开始在阴森的主屋中寻找逃脱的途径。他小心翼翼地躲避杰克的追捕，探索每个房间，寻找线索和工具，希望能找到一条通往自由的路。

(episode 4) 伊森在主屋中小心翼翼地探索，试图揭开贝克家族的秘密。阴暗的走廊和破旧的房间里充满了诡异的气息，他发现了许多令人不安的线索，揭示了这个家庭的恐怖真相。然而，伊森的每一步都充满了危险，因为杰克·贝克总是阴魂不散地出现，阻挠他的前进。杰克的疯狂和强大的力量让每次遭遇都变成了一场生死搏斗。伊森必须利用一切可以找到的武器和资源，与杰克展开多次激烈的战斗，才能在这座恐怖的主屋中生存下来。

(episode 11) 伊森在主屋中四处搜寻，试图找到更多线索。突然，他在一间不起眼的房间里发现了一块松动的地板。好奇心驱使他揭开地板，露出了一条黑暗的地下通道。伊森点亮手电，小心翼翼地沿着通道前行。通道狭窄而阴森，墙壁上布满了潮湿的苔藓和蛛网。经过一段时间的步行，他终于到达了尽头，眼前出现了一扇厚重的铁门。推开铁门，伊森惊讶地发现自己进入了一个秘密实验室。实验室里摆满了各种奇怪的仪器和化学药品，墙上挂着一些令人不安的照片和文件。伊森意识到，这个实验室隐藏着关于贝克家族和他们背后秘密的关键线索。

(episode 12) 伊森在探索贝克家族的庄园时，意外发现了一个隐藏的秘密实验室。这个实验室位于庄园的地下，充满了各

In [ ]:
# 评估plot的prompt
# prompt_evaluate_episode=PromptTemplate(
#     template = """
#         请根据总体章节的信息严格评价当前章节的目标，场景和剧情摘要，并说明原因（尽可能严格，寻找问题与不足之处）
#         可以参考但不限于以下内容：
#         1. 当前章节的目标，背景和剧情是否按照总目标前进
#         2. 是否总是以主角{leading_role}作为展开
#         3. 是否符合人物总体的行事风格以及观念
#         4. 剧情展开是否合理，有没有逻辑漏洞或者难以理解的发展过程
#         5. 剧情是否符合故事的总体风格


#         用{language}回答
#         以dict的json格式重构,格式如下: "main":{{"episode_goal":"**", "episode_scene":"**", "episode_desc":"**"}} '(**是对应的具体评价及原因)
#         需要评估的原章节内容如下：
#         原章节的目标: {episode_goal}; 原章节的场景: {episode_scene}; 原章节的剧情摘要: {episode_desc},
#         提供的总体故事背景如下：
#         故事的总体风格如下: {story_style}; 故事的总体背景如下: {story_desc}; 故事的主要目标如下: {story_goal}; 故事的人物介绍如下: {story_chars}

#     """,
#     input_variables=["leading_role","language","episode_goal","episode_scene","episode_desc","story_style","story_desc","story_goal","story_chars"],
# )

prompt_evaluate_trigger=PromptTemplate(
    template = """请根据总体章节信息严格评价当前章节跳转到下一情节的原因及跳转事件。评价时应仅关注触发器的跳转条件<IF>和后续情节<THEN>，忽略描写的生动性。
        评价标准包括但不限于：
        <IF>是否在当前情节之中可以达成，但是发生在当前情节之后
        <IF>是否仅为达成条件而非事件叙述
        <THEN>的内容是否在下一情节之前，是否是导致下一情节的关键内容
        trigger是否起到承上启下的作用
        跳转事件是否合理、符合逻辑，与故事情节及人物行为风格一致
        两段情节通过当前跳转事件连接是否流畅自然
        触发事件是否与主角{leading_role}相关（包括主角的语言或行为使其他角色触发某一条件也算是主角的原因，而非其他角色主动触发条件）
        
        请严格寻找问题与不足，展示思考过程，并提供trigger的改写办法（保持当前格式，不修改章节内容），精简回答
        改写时需注意：
        <IF>只能是一个触发原因，是当前情节之后发生的事情，不要出现当前情节发生过的内容
        <THEN>只能是一个事件，而且是下一章节发生之前的内容，不要与下一情节重复，应该能与下一章节直接衔接
        用{language}回答，不要复述输入，只需要介绍当前trigger的不足之处，并说明原因，并提供改进办法
        当前章节摘要如下: {episode_desc}, 下一章节摘要如下: {next_episode_desc}, 跳转事件如下: {trigger_condition}
        故事的总体风格如下: {story_style}; 故事的总体背景如下: {story_desc}; 故事的人物介绍如下: {story_chars}

        输出使用json格式：
        {{
            "思考过程":"(这里展示你的思考过程)",
            "不足":"(原trigger的不足)",
            "改进方法":"(提出你的改进方法)",
            "IF":"(改进后的if)",
            "THEN":"(改进后的THEN)"
        }}
    """,
    input_variables=["leading_role","language","episode_desc","next_episode_desc","trigger_condition","story_style","story_desc","story_goal","story_chars"],
)


In [ ]:

for j in tqdm(range(len(Episodes))):
    episode = Episodes[j]

    # 判断总体章节信息
    if False:
        chain_evaluate_episode = prompt_evaluate_episode | llm_agent
        res_evaluate_episode = chain_evaluate_episode.invoke({"leading_role":leading_role,"language":story_language,"episode_goal":episode["episode_goal"],"episode_scene":episode["episode_scene"],"episode_desc":episode["episode_desc"],"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars}).content
        try:
            cleaned_json_str0 = res_evaluate_episode.split('```json')[1].split('```')[0].strip()
        except:
            cleaned_json_str0 = res_evaluate_episode
        #print(cleaned_json_str0)
        res0_dict = {}
        res0_dict = json.loads(cleaned_json_str0)["main"]
            
        # print(json.dumps(res0_dict,indent=2,ensure_ascii=False))
        story_dict_new["Episodes"][j]["evaluate"] = res0_dict


    # 判断triggers
    
    triggers = episode["triggers"]
    if (not triggers == []) and (not triggers == [{}]):
        for k in range(len(triggers)):
            trigger = triggers[k]
            # 这里看一下要不要-1
            next_episode = Episodes[trigger["next_episode"]-1]
            
     
            chain_evaluate_trigger = prompt_evaluate_trigger | llm_agent
            res_evaluate_trigger = chain_evaluate_trigger.invoke({"leading_role":leading_role,"language":story_language,"episode_desc":episode["episode_desc"],"next_episode_desc":next_episode["episode_desc"],"trigger_condition":trigger["condition"],"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars}).content

            # print("当前情节: ", episode["episode_desc"])
            # print("下一情节: ", next_episode["episode_desc"])
            # print("trigger: ", trigger["condition"])
            # print(res_evaluate_trigger)

            story_dict_new["Episodes"][j]["triggers"][k]["evaluate"] = res_evaluate_trigger
            
            story_dict_new["Episodes"][j]["triggers"][k]["old_con"] = trigger["condition"]
            new_trigger_dict = res_evaluate_trigger.split('```json')[1].split('```')[0].strip("\n").strip("")
            new_trigger_dict = eval(new_trigger_dict)
            new_trigger_con = "<IF>" + new_trigger_dict["IF"] + " <THEN>" + new_trigger_dict["THEN"]
            story_dict_new["Episodes"][j]["triggers"][k]["condition"] = new_trigger_con

            # print(json.dumps( story_dict_new["Episodes"][j]["triggers"][k],indent=2,ensure_ascii=False))

            # break
            
    # break
# print(Settings)


story_dict_list_new[0] = story_dict_new
#story_dict_list_new[i] = story_dict_new

  0%|          | 0/31 [00:00<?, ?it/s]

当前情节:  伊森·温特斯的妻子米娅失踪已经三年了，突然有一天，他收到了一封来自她的求救信。信中，米娅告诉他，她被困在路易斯安那州的贝克家族农场。尽管心中充满疑虑和不安，伊森还是决定前往那个神秘而危险的地方，寻找他心爱的妻子。带着希望和恐惧，他踏上了这段未知的旅程。
下一情节:  伊森踏入贝克家族的废弃农场，四周弥漫着腐朽的气息。他在一间阴暗的地下室里找到了失踪多年的妻子米娅，但她的行为异常，仿佛被某种力量控制。正当伊森试图带她逃离时，米娅突然发狂，攻击了他。伊森奋力抵抗，最终将她制服。然而，贝克家族的成员突然出现，将伊森打昏并拖走，黑暗再次笼罩了他的视线。
trigger:  <IF> 伊森·温特斯收到失踪三年的妻子米娅的求救信，并决定前往路易斯安那州的贝克家族农场寻找她 <THEN> 伊森到达贝克家族的废弃农场，进入农场后发现米娅被囚禁并且行为异常，随后遭到贝克家族成员的袭击，剧情向下一个情节发展
```json
{
    "思考过程": "当前trigger的IF部分包含了伊森收到求救信并决定前往贝克家族农场的内容，这虽然是一个触发原因，但冗长且包含了当前情节的内容。而THEN部分直接描述了伊森到达农场后的整个事件进程，过于详细且与下一章节的内容有重复。触发器应该起到承上启下的作用，简洁明了地连接当前情节和下一情节。为了使两段情节连接更加流畅自然，需要优化IF和THEN部分，使其更加简洁，并确保THEN部分是导致下一情节的关键内容。",
    "不足": "IF部分过于冗长，包含了当前情节的内容。THEN部分过于详细，与下一章节内容有重复，未能简洁明了地起到承上启下的作用。",
    "改进方法": "简化IF部分，使其仅包含收到求救信后的决定。简化THEN部分，使其描述伊森到达农场并准备进入农场寻找米娅。",
    "IF": "伊森·温特斯决定前往路易斯安那州的贝克家族农场寻找失踪的妻子米娅",
    "THEN": "伊森到达贝克家族的废弃农场，准备进入农场寻找米娅"
}
```
{'思考过程': '当前trigger的IF部分包含了伊森收到求救信并决定前往贝克家族农场的内容，这虽然是一个触发原因，但冗长且包含了当前情节的内容。而THEN部分直接描述了伊森到达农场后的整个事件进程，过于详细且与下一章节的内容有重复。触发器应该起到承上启下的作用，

In [10]:
print(triggers)

[{}]


In [1]:
print(json.dumps(story_dict_list_new[0],indent=2,ensure_ascii=False))

NameError: name 'json' is not defined

In [12]:

game_single_file_path = data_plot_file.replace(".jsonl","_eva.jsonl")
with jsonlines.open(game_single_file_path, mode='w') as writer:
    for story_dict in story_dict_list_new:
        writer.write(story_dict)